In [1]:
import tensorflow as tf


2023-03-05 03:11:47.356426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 03:11:49.039784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/labeeb/miniconda3/pkgs/cudnn-8.4.1.50-hed8a83a_0/lib:/home/labeeb/miniconda3/pkgs/cudatoolkit-11.8.0-h37601d7_11/lib:/home/labeeb/miniconda3/envs/ee/lib/python3.10/site-packages/nvidia/cuda_runtime/lib:/usr/local/cuda/lib64:/home/labeeb/miniconda3/lib/:/usr/local/cuda/lib64:/home/labeeb/miniconda3/lib/
2023-03-05 03:11:49.040040: W tensorflow/compiler/xla/stream_executor/platform/default/dso_

In [2]:
!pip install matplotlib

/bin/bash: /home/labeeb/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
image_size=[224,224]

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
train_path='faces/train'
test_path='faces/test'

In [4]:
vgg = VGG16(input_shape=image_size+[3],weights='imagenet',include_top=False)

2023-03-05 03:12:02.659565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 03:12:02.770411: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 03:12:02.770676: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-05 03:12:02.771388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [5]:
for layer in vgg.layers : 
    layer.trainable=False

In [6]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(16, activation='relu')
prediction_layer = layers.Dense(2, activation='softmax')


model = models.Sequential([
    vgg,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 50)                1254450   
                                                                 
 dense_1 (Dense)             (None, 16)                816       
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 15,969,988
Trainable params: 1,255,300
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    directory=r"data/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 13000 images belonging to 2 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = train_datagen.flow_from_directory(
    directory=r"data/test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 1000 images belonging to 2 classes.


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
r=model.fit(train_generator,validation_data=test_generator,epochs=1,steps_per_epoch=len(train_generator),validation_steps=len(test_generator))

2023-03-05 03:13:17.478444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-03-05 03:13:19.196697: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-05 03:13:19.196734: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-03-05 03:13:19.196747: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

 19/407 [>.............................] - ETA: 3:28 - loss: 0.4402 - accuracy: 0.8487

/home/labeeb/miniconda3/lib/python3.10/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


407/407 [==============================] - 206s 489ms/step - loss: 0.0575 - accuracy: 0.9824 - val_loss: 0.0104 - val_accuracy: 0.9970


In [14]:
img = 'cartoon.png'

In [13]:
import numpy as np
import os

def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor


In [27]:
img = 'input1.jpeg'
img = load_image(img)

In [28]:
pred = model.predict(img)

1/1 [==============================] - 0s 15ms/step


In [29]:
pred

array([[9.999999e-01, 1.359676e-07]], dtype=float32)

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [31]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpo_a25gqi/assets


INFO:tensorflow:Assets written to: /tmp/tmpo_a25gqi/assets
2023-03-05 03:19:40.159802: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-03-05 03:19:40.159840: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-03-05 03:19:40.160703: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpo_a25gqi
2023-03-05 03:19:40.166183: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-03-05 03:19:40.166205: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpo_a25gqi
2023-03-05 03:19:40.180196: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-05 03:19:40.184046: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-03-05 03:19:40.309917: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

In [32]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [33]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets
